## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torchvision
import pickle
import torch.nn as nn
from helpers import *
import urllib
from visdom import Visdom
from torch._jit_internal import weak_module,weak_script_method
import matplotlib.pyplot as plt
#viz = Visdom(env='Testouiles')
#from torch.nn.backends.backend.j import weak_module, weak_script_method

In [3]:
@weak_module
class SpatialPyramidPooling(nn.Module) :
    def __init__(self,pyramid_levels=3) : 
        super(SpatialPyramidPooling,self).__init__()
        self.pyramid = [nn.AdaptiveMaxPool2d(output_size=(1,1))]
        for i in range(1,pyramid_levels) : 
            self.pyramid.append(nn.AdaptiveMaxPool2d(output_size=(i*2,i*2)))
        
    @weak_script_method
    def forward(self,input) :
        cat = []
        for p in self.pyramid : 
            cat.append(p(input).view(input.size(0),-1))
        return torch.cat(cat,dim=1)

## Load Datas

In [4]:
dataloaders, dataset_sizes, device,class_names,image_datasets = load_datas('../bruvs')

In [5]:
batch = next(iter(dataloaders))
batch_gpu = batch[0].to(device)
#viz.images(unormalize_batch(batch_gpu,mean,std),win='expample images',opts=dict(title='Shark images'))

In [19]:
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16.features[-1] = SpatialPyramidPooling()
classifier = nn.Sequential(nn.Linear(10752,2048,bias=True),nn.ReLU(),nn.Linear(2048,1),nn.Sigmoid())
shark_detector = nn.Sequential(vgg16.features,classifier)
shark_detector.to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [20]:
t = shark_detector(batch_gpu)

In [21]:
t

tensor([[0.5123],
        [0.4945],
        [0.5315],
        [0.5150],
        [0.5171],
        [0.4840],
        [0.5244],
        [0.5151],
        [0.5087],
        [0.5106]], device='cuda:0', grad_fn=<SigmoidBackward>)

# View imagenet Prediction

In [9]:
class_imagenet = pickle.load(urllib.request.urlopen('https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl') )

In [12]:
text_preds = ['{} {}'.format(class_imagenet[x.item()].split(',')[0],p) for p,x in list(zip(list(predictions.max(dim=1))[0],list(predictions.max(dim=1))[1]))]
text_preds

['hammerhead 10.220370292663574',
 'hammerhead 9.88038158416748',
 'hammerhead 9.702723503112793',
 'hammerhead 9.298279762268066',
 'hammerhead 10.194511413574219',
 'hammerhead 10.02170467376709',
 'scuba diver 8.803277969360352',
 'scuba diver 10.63332748413086',
 'hammerhead 8.901375770568848',
 'hammerhead 8.357734680175781']